# Extracting commonsense knowledge from Wikidata

## Define relevant properties

In [ ]:
%env properties="P1889,P461,P527,P186,P463,P276,P170,P366,P279,P1552,P828,P1542,P462" #,P31"

## Filter relevant properties

In [ ]:
%%bash
kgtk filter -p " ; $properties ; " input/wikidata/wikidata_edges_20200504.tsv.gz > tmp/kgtk_wikidata_filter.tsv

## Remove columns

In [47]:
%env ignore_cols=id,rank,node2;magnitude,node2;unit,node2;item,node2;lower,node2;upper,node2;entity-type,node2;longitude,node2;latitude,node2;date,node2;calendar,node2;precision

env: ignore_cols=id,rank,node2;magnitude,node2;unit,node2;item,node2;lower,node2;upper,node2;entity-type,node2;longitude,node2;latitude,node2;date,node2;calendar,node2;precision


In [48]:
%%bash
kgtk remove_columns -c "$ignore_cols" -i tmp/kgtk_wikidata_filter.tsv > tmp/kgtk_wikidata_cols.tsv

## Deduplicate

In [49]:
%%bash
kgtk compact -i tmp/kgtk_wikidata_cols.tsv -o tmp/kgtk_wikidata_compact.tsv

## Add labels

In [50]:
%%bash
kgtk --debug lift --verbose \
     --input-file tmp/kgtk_wikidata_compact.tsv \
     --label-file input/wikidata/wiki_labels.tsv \
     --output-file tmp/kgtk_wikidata.tsv \
     --columns-to-lift node1 node2 label \
     --prefilter-labels

Opening the input file: tmp/kgtk_wikidata_compact.tsv
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file tmp/kgtk_wikidata_compact.tsv
header: node1	label	node2
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=-1
KgtkReader: Reading an edge file.
Opening the label file: input/wikidata/wiki_labels.tsv
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file input/wikidata/wiki_labels.tsv
header: node1	label	node2
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=-1
KgtkReader: Reading an edge file.
Lifting with in-memory buffering.
Reading input data to prefilter the labels.
Loading input rows without labels from tmp/kgtk_wikidata_compact.tsv
Labels needed: 5611919
Loading labels from the label file.
Loading labels from input/wikidata/wiki_labels.tsv
Filtering for needed labels
label_match_column_idx=0 (node1).
label_select_column_idx=1 (label).
label_value_column_idx=2 (no

## Add PageRank

In [51]:
%%bash
kgtk --debug lift --verbose \
     --input-file tmp/kgtk_wikidata.tsv \
     --label-file input/wikidata/wikidata-pagerank-only-sorted2.tsv \
     --output-file tmp/kgtk_wikidata_with_pr.tsv \
     --columns-to-lift node1 node2 \
     --property vertex_pagerank \
     --lift-suffix ";pagerank" \
     --prefilter-labels

Opening the input file: tmp/kgtk_wikidata.tsv
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file tmp/kgtk_wikidata.tsv
header: node1	label	node2	node1;label	node2;label	label;label
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=-1
KgtkReader: Reading an edge file.
Opening the label file: input/wikidata/wikidata-pagerank-only-sorted2.tsv
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file input/wikidata/wikidata-pagerank-only-sorted2.tsv
header: node1	relation	node2	id
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=0 label=1 node2=2 id=3
KgtkReader: Reading an edge file.
Lifting with in-memory buffering.
Reading input data to prefilter the labels.
Loading input rows without labels from tmp/kgtk_wikidata.tsv
Labels needed: 5611908
Loading labels from the label file.
Loading labels from input/wikidata/wikidata-pagerank-only-sorted2.tsv
Filtering for needed labels
label_match_column_idx=

## Filter concepts vs instances

In [52]:
import pandas as pd
df=pd.read_csv('tmp/kgtk_wikidata_with_pr.tsv', sep='\t')

In [53]:
len(df)

8256446

In [54]:
df.head()

,node1,label,node2,node1;label,node2;label,label;label,node1;pagerank,node2;pagerank
0,P1005,P1552,Q26921380,'Portuguese National Library ID'@en,'VIAF component'@en,'has quality'@en,3.630733e-09,0.000004
1,P1006,P1552,Q26921380,'Nationale Thesaurus voor Auteurs ID'@en,'VIAF component'@en,'has quality'@en,7.545395e-08,0.000004
2,P1015,P1552,Q26921380,'NORAF ID'@en,'VIAF component'@en,'has quality'@en,3.637108e-09,0.000004
3,P1017,P1552,Q26921380,'Vatican Library ID'@en,'VIAF component'@en,'has quality'@en,2.345845e-07,0.000004
4,P1048,P1552,Q26921380,'NCL ID'@en,'VIAF component'@en,'has quality'@en,2.483013e-09,0.000004


In [55]:
limit=1.419947235126343e-08 # PageRank of the 752876th most popular node

In [56]:
cols=['node1;pagerank', 'node2;pagerank']
cols=['node1;label', 'node2;label']

In [57]:
from tqdm import tqdm

In [77]:
new_rows=[]
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    store=True
    if row['node1;label']=='' or row['node2;label']=='' or not isinstance(row['node1;label'], str) or not isinstance(row['node2;label'], str): continue
    node1_label=row['node1;label'].strip()[1:-3]
    node2_label=row['node2;label'].strip()[1:-3]
    if node1_label[0].islower() and node2_label[0].islower():
        a_row=[*row[:6], "", "", "WD", "", "", ""] # copy id, node1, relation, node2, node1:label, node2:label, relation:label
        a_row[3]=node1_label
        a_row[4]=node2_label
        new_rows.append(a_row)

  0%|          | 1333/8256446 [00:02<2132:21:02,  1.08it/s]

yo


100%|██████████| 8256446/8256446 [18:23<00:00, 7478.68it/s]  


## Filter by usage

In [94]:
from wordfreq import word_frequency

In [112]:
threshold=1e-06

In [113]:
len(new_rows)

1255113

In [114]:
filtered_rows=[]
for a_row in new_rows:
    node1_label=a_row[3]
    node2_label=a_row[4]
    wf1=word_frequency(node1_label, 'en')
    wf2=word_frequency(node2_label, 'en')
    if wf1 >threshold and wf2>threshold:
        filtered_rows.append(a_row)

In [115]:
len(filtered_rows)

207798

In [116]:
new_columns=['node1', 'relation', 'node2', 'node1;label', 'node2;label','relation;label', 'relation;dimension', 'weight', 'source', 'origin', 'sentence', 'question']

In [117]:
df2=pd.DataFrame(filtered_rows, columns=new_columns)

In [118]:
df2['relation'].value_counts()

P279     187128
P527       8153
P461       3114
P366       3071
P186       2358
P1552      1773
P828        868
P1542       778
P170        368
P276        170
P463         17
Name: relation, dtype: int64

In [119]:
df2.to_csv('tmp/kgtk_wikidata_ready.tsv', index=False, sep='\t', columns=new_columns)

In [135]:
with pd.option_context('display.max_rows', None, 'display.width', 1000):  # more options can be specified also
    print(df2[df2['relation']=='P186'].head(100))

          node1 relation      node2            node1;label             node2;label      relation;label relation;dimension weight source origin sentence question
149    Q1007164     P186     Q12117          grain whisky'                 cereal'  'material used'@en                               WD                         
220    Q1018244     P186     Q11469           crown glass'                  glass'  'material used'@en                               WD                         
424      Q10289     P186     Q11427                barrel'                  steel'  'material used'@en                               WD                         
425      Q10289     P186     Q11474                barrel'                plastic'  'material used'@en                               WD                         
426      Q10289     P186       Q287                barrel'                   wood'  'material used'@en                               WD                         
475    Q1030197     P186     Q1142

## Compute statistics

In [84]:
%%bash
kgtk graph_statistics --directed --degrees --pagerank --hits --log summary.txt -i tmp/kgtk_wikidata_ready.tsv > tmp/stats/wiki_stats.tsv

In [85]:
%%bash
cat summary.txt

loading the TSV graph now ...
graph loaded! It has 821468 nodes and 1255113 edges

###Top relations:
P279	1189215
P527	38663
P186	7220
P366	5793
P461	5571
P1552	3330
P828	2149
P1542	1881
P170	698
P276	566

###Degrees:
in degree stats: mean=1.527890, std=0.616167, max=1
out degree stats: mean=1.527890, std=0.000787, max=1
total degree stats: mean=3.055781, std=0.616171, max=1

###PageRank
Max pageranks
436437	Q50365914	0.025876
11480	Q35120	0.034357
6464	Q8054	0.037386
7899	Q7187	0.060852
55207	Q20747295	0.035241

###HITS
HITS hubs
55194	Q201448	0.000024
74109	Q277338	0.000174
6464	Q8054	0.000044
7899	Q7187	0.694359
55207	Q20747295	0.719629
HITS auth
408217	Q24178445	0.001748
408218	Q24178446	0.001748
408213	Q24178441	0.001748
408214	Q24178442	0.001748
410733	Q24181279	0.001748
